In [6]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row
from bokeh.io import push_notebook
from math import sqrt

output_notebook()

In [7]:
class DiffEq:
    def __init__(self, de="sqrt(y-x)/sqrt(x)+1", x0=1, y0=10, X=15, n=10):
        self.de = de
        self.x0 = x0
        self.y0 = y0
        self.X = X
        self.n = n
        self.h = (self.X - self.x0) / self.n
        self.x_arr = [self.x0]
        self.y_arr = [self.y0]
        self.y_arr_LTE = [self.y0]
        self.y_arr_GTE = [self.y0]
        for i in range(1, self.n + 1):
            self.x_arr.append(self.x_arr[i - 1] + self.h)

    def diff_eq(de, x, y):
        return eval(de)

In [19]:
class EulerMethod(DiffEq):
  def solve(self):
    for i in range(1,self.n+1):
      y_i=self.y_arr[i-1]+self.h*DiffEq.diff_eq(self.de, self.x_arr[i-1],self.y_arr[i-1])
      self.y_arr.append(y_i)
    return self.y_arr
  def LTE(self,y_exact):
    for i in range(1,self.n+1):
      y_i=y_exact[i-1]+self.h*DiffEq.diff_eq(self.de,self.x_arr[i-1],y_exact[i-1])
      self.y_arr_LTE.append(y_i)
    euler_LTE=[abs(x - y) for x, y in zip(y_exact, self.y_arr_LTE)]
    return euler_LTE

In [9]:
class ImprovedEulerMethod(DiffEq):
  def solve(self):
    for i in range(1,self.n+1):
      x_val=self.x_arr[i-1]+(self.h/2)
      y_val=self.y_arr[i-1]+(self.h/2)*DiffEq.diff_eq(self.de, self.x_arr[i-1],self.y_arr[i-1])
      y_i=self.y_arr[i-1]+self.h*DiffEq.diff_eq(self.de, x_val,y_val)
      self.y_arr.append(y_i)
    return self.y_arr
  def LTE(self,y_exact):
    for i in range(1,self.n+1):
      x_val=self.x_arr[i-1]+(self.h/2)
      y_val=y_exact[i-1]+(self.h/2)*DiffEq.diff_eq(self.de, self.x_arr[i-1],y_exact[i-1])
      y_i=y_exact[i-1]+self.h*DiffEq.diff_eq(self.de, x_val,y_val)
      self.y_arr_LTE.append(y_i)
    improvedEuler_LTE=[abs(x - y) for x, y in zip(y_exact, self.y_arr_LTE)]
    return improvedEuler_LTE

In [20]:
class RungeKuttaMethod(DiffEq):
  def solve(self):
    for i in range(1,self.n+1):
      k1=DiffEq.diff_eq(self.de, self.x_arr[i-1],self.y_arr[i-1])
      k2=DiffEq.diff_eq(self.de, self.x_arr[i-1]+(self.h/2),self.y_arr[i-1]+(self.h*k1)/2)
      k3=DiffEq.diff_eq(self.de, self.x_arr[i-1]+(self.h/2),self.y_arr[i-1]+(self.h*k2)/2)
      k4=DiffEq.diff_eq(self.de, self.x_arr[i-1]+self.h,self.y_arr[i-1]+self.h*k3)
      y_i=self.y_arr[i-1]+(self.h/6)*(k1+2*k2+2*k3+k4)
      self.y_arr.append(y_i)
    return self.y_arr
  def LTE(self,y_exact):
    for i in range(1,self.n+1):
      k1=DiffEq.diff_eq(self.de, self.x_arr[i-1],y_exact[i-1])
      k2=DiffEq.diff_eq(self.de, self.x_arr[i-1]+(self.h/2),y_exact[i-1]+(self.h*k1)/2)
      k3=DiffEq.diff_eq(self.de, self.x_arr[i-1]+(self.h/2),y_exact[i-1]+(self.h*k2)/2)
      k4=DiffEq.diff_eq(self.de, self.x_arr[i-1]+self.h,y_exact[i-1]+self.h*k3)
      y_i=y_exact[i-1]+(self.h/6)*(k1+2*k2+2*k3+k4)
      self.y_arr_LTE.append(y_i)
      RK_LTE=[abs(x - y) for x, y in zip(y_exact, self.y_arr_LTE)]
    return RK_LTE

In [11]:
class Exact(DiffEq):
    def __init__(self, de="sqrt(y-x)/sqrt(x)+1", x0=1, y0=10, X=15, n=10,
                 y_sol="x*(1+(1+(sqrt(y0-x0)-sqrt(x0))/sqrt(x))**2)"):
        DiffEq.__init__(self, de, x0, y0, X, n)
        self.y_sol = y_sol

    def solve(self):
        x0 = self.x0
        y0 = self.y0
        for i in range(1, self.n + 1):
            x = self.x_arr[i]
            self.y_arr.append(eval(self.y_sol))
            x = self.x_arr[i]
        return self.y_arr

In [12]:
#Input data
def Input():
    x0=int(input("Input x0: "))
    assert x0>0, "Error: x0 must be greater than zero"
    y0=int(input("Input y0: "))
    assert y0>0, "Error: y0 must be greater than zero"
    assert y0>x0, "Error: y0 must be greater than x0"
    X=int(input("Input X: "))
    assert X>0, "Error: X must be greater than zero"
    assert X>x0, "Error: X must be greater than x0"
    N=int(input("Input N: "))
    assert N>0, "Error: N must be greater than zero"
    N_0=int(input("Input starting number for graph of errors: "))
    assert N_0>0, "Error: N must be greater than zero"
    N_n=int(input("Input finishing number for graph of errors: "))
    assert N_n>0, "Error: N must be greater than zero"

In [39]:
#Graph of each method and exact solution
def PlottGraph(x_arr, euler,IE,RK,exact):
  graph=figure(title="Methods",x_axis_label="x", y_axis_label='y')

  graph.line(x_arr, euler, line_width =1, color="blue", legend_label = "Euler method")
  graph.line(x_arr, IE, line_width =1, color="red", legend_label = "Improved Euler method")
  graph.line(x_arr, RK, line_width =1, color="green", legend_label = "Runge-Kutta method")
  graph.line(x_arr, exact, line_width =1, color="black", legend_label = "Exact solution")

  graph.legend.click_policy = "hide"
  show(graph)

In [40]:
#Graph of LTE or GTE
def PlottLGTEGraph(x_arr, euler,IE,RK,exact, name="no name"):

  graphLTE=figure(title=name,x_axis_label="x", y_axis_label='y')

  graphLTE.line(x_arr, euler, line_width =1, color="blue", legend_label = "Euler method")
  graphLTE.line(x_arr, IE, line_width =1, color="red", legend_label = "Improved Euler method")
  graphLTE.line(x_arr, RK, line_width =1, color="green", legend_label = "Runge-Kutta method")

  graphLTE.legend.click_policy = "hide"
  show(graphLTE)


In [43]:
#Calculationg and plotting GTE from N_0 to N_n
def PlottGTEGraph_n0_n(N_0 = 10,N_n = 100):
  euler_GTE = []
  IE_GTE = []
  RK_GTE = []
  x_GTE = []
  
  #Calculating
  for i in range(N_0, N_n + 1):
    x_GTE.append(i)
  for i in range(N_0, N_n + 1):
    ex = Exact(n=i)
    ex_results = ex.solve()

    eu = EulerMethod(n=i)
    eu_results = eu.solve()

    IE = ImprovedEulerMethod(n=i)
    IE_results = IE.solve()

    RKM = RungeKuttaMethod(n=i)
    RKM_results = RKM.solve()

    euler_GTE.append(max([abs(x - y) for x, y in zip(ex_results, eu_results)]))
    IE_GTE.append(max([abs(x - y) for x, y in zip(ex_results, IE_results)]))
    RK_GTE.append(max([abs(x - y) for x, y in zip(ex_results, RKM_results)]))
  
  #Plotting
  string="GTE for step = (from {} to {})".format(N_0,N_n)
  PlottLGTEGraph(x_GTE, euler_GTE,IE_GTE,RK_GTE,exact, string)


In [ ]:
#Input data
#Input()

In [22]:
#Calculating solution and methods
euler=EulerMethod()
euler_results=euler.solve()

improvedEuler=ImprovedEulerMethod()
improvedEuler_results=improvedEuler.solve()

RK=RungeKuttaMethod()
RK_results=RK.solve()

exact=Exact()
exact_results=exact.solve()

In [23]:
#Calculating LTE
euler_LTE=euler.LTE(exact_results)
improvedEuler_LTE=improvedEuler.LTE(exact_results)
RK_LTE=RK.LTE(exact_results)

In [46]:
#Calculating GTE
euler_GTE=[abs(x - y) for x, y in zip(exact_results, euler_results)]
improvedEuler_GTE=[abs(x - y) for x, y in zip(exact_results, improvedEuler_results)]
RK_GTE=[abs(x - y) for x, y in zip(exact_results, RK_results)]

In [47]:
PlottGraph(euler.x_arr, euler_results,improvedEuler_results,RK_results,exact_results)
PlottLGTEGraph(euler.x_arr, euler_LTE,improvedEuler_LTE,RK_LTE,exact_results, "LTE")
PlottLGTEGraph(euler.x_arr, euler_GTE,improvedEuler_GTE,RK_GTE,exact_results, "GTE")
PlottGTEGraph_n0_n()